# Importando bibliotecas e etc

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_datareader.data as web

!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
yf.pdr_override()

     |████████████████████████████████| 6.3 MB 7.5 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
#carteira = web.get_data_yahoo('BOVA11.SA', period='20y')
carteira = web.get_data_yahoo('^BVSP')[['Open','High', 'Low', 'Close']].div(1000, axis=1)
carteira

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
1993-04-27,0.024800,0.025400,0.024500,0.024500
1993-04-28,0.024500,0.024600,0.023700,0.024300
1993-04-29,0.024300,0.024800,0.023700,0.023700
1993-04-30,0.023700,0.024200,0.023700,0.024100
1993-05-03,0.024100,0.024400,0.023800,0.024100
...,...,...,...,...
2021-11-29,102.227000,104.149000,102.226000,102.814000
2021-11-30,102.814000,103.066000,100.075000,101.915000
2021-12-01,101.916000,104.087000,100.727000,100.775000


In [ ]:
#carteira = carteira.loc['2005-08-01':'2021-07-30']
#carteira['Variação'] = carteira['Close'].pct_change()
#carteira.dropna(inplace=True)

# **Simulação ideal**

### Determina o melhor cenário para investimento: aquele em que o preço da ação é o menor possível para o mês

##### Este bloco determina os dias dos meses com os menores preços

In [ ]:
index_dias = []
index_data_ano = carteira.index.year.drop_duplicates(keep='first')
index_data_mês = carteira.index.month.drop_duplicates(keep='first')
for ano in index_data_ano:
  for mês in index_data_mês:
    index_dias.append(carteira['Low'].where(carteira.index.year==ano).where(carteira.index.month==mês).idxmin())

index_dias = pd.DataFrame({'Data':index_dias})
index_dias = index_dias.dropna()
index_dias

,Data
0,1993-04-28
1,1993-05-03
2,1993-06-01
3,1993-07-02
4,1993-08-05
...,...
343,2021-11-30
344,2021-12-01
345,2021-01-27
346,2021-02-26


#### Este bloco calcula o custo total de investimento no cenário ideal

In [ ]:
soma = []
montante = []
gasto = []
for c in index_dias['Data']:
  gasto.append(carteira['Low'][c])

index_dias.set_index('Data', inplace=True)
index_dias['Gasto'] = gasto
index_dias['Gasto_total'] = index_dias['Gasto'].cumsum()

index_dias

,Gasto,Gasto_total
Data,,
1993-04-28,0.0237,0.0237
1993-05-03,0.0238,0.0475
1993-06-01,0.0354,0.0829
1993-07-02,0.0517,0.1346
1993-08-05,0.0606,0.1952
...,...,...
2021-11-30,100.0750,13283.3214
2021-12-01,100.7270,13384.0484
2021-01-27,114.8870,13498.9354


#### Calcula o montante, lucro e lucro percentual ao final do período com respeito ao cenário ideal

In [ ]:
montante = carteira['Close'][-1] * len(index_dias)
lucro_ideal = montante - index_dias['Gasto_total'][-1]
lucro_percentual_ideal = 100*lucro_ideal/index_dias['Gasto_total'][-1]

print(lucro_ideal, lucro_percentual_ideal)

22607.060318851472 164.8215671789373


# **Simulação aleatória**


In [ ]:
import random
import statistics as st
gasto_total, lucro, lucro_percentual, luc_dif = [], [], [], []

carteira_df = carteira.copy()   #Faz uma cópia da carteira
carteira_df = carteira_df.reset_index() #Reseta os índices: agora os índices serão números
carteira_df['month'] = carteira_df['Date'].astype('datetime64[M]') #Cria uma nova coluna chamada "month" com dia fixo em 1 e variando mês e ano

for i in range(2):
  # Captura aleatoriamente as datas que serão realizados os investimentos
  df_esc = carteira_df.groupby(['month'])['Low'].sample().reset_index(name="sort")
  gasto =  []
  for c in df_esc['index']:
    a = carteira_df['Low'][c]
    b = carteira_df['High'][c]
    gasto.append(random.uniform(a,b))

  gasto_total.append(sum(gasto))
  lucro.append((carteira['Close'][-1]*len(df_esc)) - gasto_total[-1]) 
  lucro_percentual.append(100*(lucro[-1]/gasto_total[-1]))

lucro_medio = st.mean(lucro)
lucro_minimo = min(lucro)
lucro_maximo = max(lucro)
lucro_std = st.stdev(lucro) # desvio padrão

lucro_per_medio = st.mean(lucro_percentual)
lucro_per_min = min(lucro_percentual)
lucro_per_max = max(lucro_percentual)

print(lucro_medio, lucro_minimo, lucro_maximo, lucro_std, lucro_per_medio, lucro_per_min, lucro_per_max)
#print(st.mean(luc_dif))

21779.86336129709 21773.77678699207 21785.94993560211 8.60771593055073 149.75901023236224 149.65448244043228 149.8635380242922


# Parâmetro IdealxAleatório



In [ ]:
df_p1 = carteira.loc['2011-08-01':'2021-07-30']
df_p2 = carteira.loc['2010-08-01':'2020-07-30']
df_p3 = carteira.loc['2009-08-01':'2019-07-30']
df_p4 = carteira.loc['2008-08-01':'2018-07-30']
df_p5 = carteira.loc['2007-08-01':'2017-07-30']
df_p6 = carteira.loc['2006-08-01':'2016-07-30']
df_p7 = carteira.loc['2005-08-01':'2015-07-30']

lista_df = [df_p1, df_p2, df_p3, df_p4, df_p5, df_p6, df_p7]
print(df_p1)

               Open     High      Low    Close
Date                                          
2011-08-01   58.823   59.542   58.168   58.536
2011-08-02   58.526   58.673   57.259   57.311
2011-08-03   57.318   57.352   55.249   56.017
2011-08-04   55.999   55.999   52.629   52.811
2011-08-05   52.811   53.866   51.153   52.949
...             ...      ...      ...      ...
2021-07-26  125.058  126.214  125.006  126.004
2021-07-27  126.004  126.026  123.670  124.612
2021-07-28  124.615  126.712  124.542  126.286
2021-07-29  126.285  126.476  124.917  125.675
2021-07-30  125.672  125.673  121.748  121.801

[2471 rows x 4 columns]


In [ ]:
lista_carteira_df = []
lista_df_esc = []
lista_df_ideais = []

for df in lista_df:
  carteira_df = df.copy()   #Faz uma cópia da df
  carteira_df = carteira_df.reset_index() #Reseta os índices: agora os índices serão números
  carteira_df['month'] = carteira_df['Date'].astype('datetime64[M]') #Cria uma nova coluna chamada "month" com dia fixo em 1 e variando mês e ano
  df_esc = carteira_df.groupby(['month'])['Low'].sample().reset_index(name="sort") #Cria uma coluna "sort" para alocar o valor escolhido e reseta o indice para guardar o índice do valor que foi escolhido
  df_ideais = carteira_df.groupby(['month'])['Low'].min().reset_index(name='meses')

  lista_carteira_df.append(carteira_df)
  lista_df_ideais.append(df_ideais)
  lista_df_esc.append(df_esc)

  montante = carteira_df['Close'].iloc[-1] * len(df_ideais)
  lucro_ideal = montante - sum(df_ideais['meses'])
  lucro_percentual_ideal = 100*lucro_ideal/sum(df_ideais['meses'])

  print(lucro_ideal, lucro_percentual_ideal)
  print()

6520.621000000002 80.54625168874708

5033.846000000001 66.52161146331675

5159.764000000002 71.74114832243174

3000.9849999999988 45.249154307021776

1426.3430000000026 22.171259930949915

651.8600000000015 10.47147836982541

-92.1909999999998 -1.5163414904131352



In [ ]:
import random
import statistics as st

for z in range(len(lista_df_esc)):
  gasto_total, lucro, lucro_percentual, luc_dif = [], [], [], []

  for i in range(1):
    lista_df_esc[z] = lista_carteira_df[z].groupby(['month'])['Low'].sample().reset_index(name="sort")
    lista_df_ideais[z] = lista_carteira_df[z].groupby(['month'])['Low'].min().reset_index(name='meses')
    gasto =  []
    
    for num, c in enumerate(lista_df_esc[z]['index']): 
      parametro = random.choices([0,1], [100,0])
      if parametro == [0]:
        gasto.append(lista_df_ideais[z]['meses'][num])
      if parametro == [1]:
        a = lista_carteira_df[z]['Low'][c]
        b = lista_carteira_df[z]['High'][c]
        gasto.append(random.uniform(a,b))

    gasto_total.append(sum(gasto))
    lucro.append(((lista_carteira_df[z]['Close'].iloc[-1]*len(lista_df_esc[z])) - gasto_total[i]))
    lucro_percentual.append(100*(lucro[i]/gasto_total[i]))

  lucro_medio = st.mean(lucro)
  lucro_minimo = min(lucro)
  lucro_maximo = max(lucro)
  #lucro_std = st.stdev(lucro) # desvio padrão

  lucro_per_medio = st.mean(lucro_percentual)
  lucro_per_min = min(lucro_percentual)
  lucro_per_max = max(lucro_percentual)

  print(round(lucro_medio, 2), round(lucro_minimo,2), round(lucro_maximo,2), round(lucro_std,2), round(lucro_per_medio,2), round(lucro_per_min,2), round(lucro_per_max,2))
  print()

6520.62 6520.62 6520.62 0.0 80.55 80.55 80.55

5033.85 5033.85 5033.85 0.0 66.52 66.52 66.52

5159.76 5159.76 5159.76 0.0 71.74 71.74 71.74

3000.98 3000.98 3000.98 0.0 45.25 45.25 45.25

1426.34 1426.34 1426.34 0.0 22.17 22.17 22.17

651.86 651.86 651.86 0.0 10.47 10.47 10.47

-92.19 -92.19 -92.19 0.0 -1.52 -1.52 -1.52

